In [1]:
from bs4 import BeautifulSoup # For HTML parsing
from urllib.request import urlopen # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#function to clean scraped text

def text_cleaner(website):
    '''
    Inputs: url
    Outputs: list of unique words
    '''
    try:
        site = urlopen(website).read() # Connect to the job posting
    except: 
        return  
    
    soup_obj = BeautifulSoup(site,"lxml") # Get the html from the site
     
#     for script in soup_obj(["script", "style"]):
#         script.extract() # Remove these two elements from the BS4 object
    
    mydivs = soup_obj.findAll("div", {"class":"quote"}, {"b"}) #identify job description 
    
    texts = []
    
    
    for i in range(len(mydivs)):
        text = mydivs[i].get_text().split("\n") # Get the text from job description
        texts.append(text)
    
#     for i in range(len(texts)):
#         texts[i] = ' '.join(word for word in texts[i])
    

    return texts

In [3]:
#retrieve links to script of each episode

base_url = 'http://officequotes.net/'

current_page = "http://officequotes.net/no1-01.php"
html_page = urlopen(current_page).read() # Get the page
page_obj = BeautifulSoup(html_page) # Locate all of the job links
eplinks = page_obj.find_all('a')        
eplinks= [link.get('href') for link in eplinks] # Get the URLS for the jobs
eplinks = [(base_url + str(link)) for link in eplinks]  
eplinks = [links for links in eplinks if "no" in links]

C:\Users\Johnny\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Johnny\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
#pass each script through scraping/cleaning function

scripts = []

for j in range(len(eplinks)):
# for j in range(0,3):
    final_description = text_cleaner(eplinks[j])
    if final_description: # So that we only append when the website was accessed correctly
        scripts.append(final_description)
        
print('Done with collecting the episodes')  

Done with collecting the episodes


In [6]:
#collapse lists
newlist = [item for items in scripts for item in items]
finallist = [item for items in newlist for item in items]
finallist = [ x for x in finallist if ":" in x ]

In [8]:
r = pd.DataFrame()
# r["character"] = newlist
r["character"] = finallist

In [9]:
r["character"] = r["character"].str.lower()
test = r
test['Name'], test['Line'] = test['character'].str.split(':', 1).str
test

,character,Name,Line
0,michael: all right jim. your quarterlies look ...,michael,all right jim. your quarterlies look very goo...
1,"jim: oh, i told you. i couldn't close it. so...",jim,"oh, i told you. i couldn't close it. so..."
2,michael: so you've come to the master for guid...,michael,so you've come to the master for guidance? is...
3,"jim: actually, you called me in here, but yeah.",jim,"actually, you called me in here, but yeah."
4,"michael: all right. well, let me show you how ...",michael,"all right. well, let me show you how it's done."
5,"michael: [on the phone] yes, i'd like to speak...",michael,"[on the phone] yes, i'd like to speak to your..."
6,"michael: i've, uh, i've been at dunder mifflin...",michael,"i've, uh, i've been at dunder mifflin for 12 ..."
7,pam: well. i don't know.,pam,well. i don't know.
8,"michael: if you think she's cute now, you shou...",michael,"if you think she's cute now, you should have ..."
9,pam: what?,pam,what?


In [10]:
test2 = test.iloc[:, 1:3]

In [11]:
final = test2.dropna()
final

,Name,Line
0,michael,all right jim. your quarterlies look very goo...
1,jim,"oh, i told you. i couldn't close it. so..."
2,michael,so you've come to the master for guidance? is...
3,jim,"actually, you called me in here, but yeah."
4,michael,"all right. well, let me show you how it's done."
5,michael,"[on the phone] yes, i'd like to speak to your..."
6,michael,"i've, uh, i've been at dunder mifflin for 12 ..."
7,pam,well. i don't know.
8,michael,"if you think she's cute now, you should have ..."
9,pam,what?


In [12]:
final.to_csv("OfficeNew2019.csv")